# Module 1 Homework: Docker & SQL

In this homework we'll prepare the environment and practice
Docker and SQL

When submitting your homework, you will also need to include
a link to your GitHub repository or other public code-hosting
site.

This repository should contain the code for solving the homework.

When your solution has SQL or shell commands and not code
(e.g. python files) file format, include them directly in
the README file of your repository.

## Question 1. Understanding Docker images

Run docker with the `python:3.13` image. Use an entrypoint `bash` to interact with the container.

What's the version of `pip` in the image?

- 25.3 [x]
- 24.3.1
- 24.2.1
- 23.3.1

## Question 2. Understanding Docker networking and docker-compose

Given the following `docker-compose.yaml`, what is the `hostname` and `port` that pgadmin should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

- postgres:5433
- localhost:5432
- db:5433 [x]
- postgres:5432
- db:5432

If multiple answers are correct, select any 


## Prepare the Data

Download the green taxi trips data for November 2025:

```bash
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
```

You will also need the dataset with zones:

```bash
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

--2026-01-13 22:08:12--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:20e8:2400:b:20a5:b140:21, 2600:9000:20e8:4200:b:20a5:b140:21, 2600:9000:20e8:800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:20e8:2400:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1164775 (1.1M) [binary/octet-stream]
Saving to: ‘green_tripdata_2025-11.parquet’

green_tripdata_2025 100%[===================>]   1.11M  --.-KB/s    in 0.09s   

2026-01-13 22:08:12 (12.2 MB/s) - ‘green_tripdata_2025-11.parquet’ saved [1164775/1164775]

--2026-01-13 22:08:12--  https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
Resolving github.com (github.com)... 240.0.6.47
Connecting to github.com (github.com)|240.0.6.47|:443... connected.
HTTP request sent, awaiting response...

In [2]:
import pandas as pd
import numpy as np

In [5]:
tzl_df = pd.read_csv('./taxi_zone_lookup.csv')
trip_df = pd.read_parquet('./green_tripdata_2025-11.parquet')

In [6]:
trip_df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,None,NaN,59,51,NaN,8.50,33.22,...,0.5,0.00,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,None,NaN,74,151,NaN,1.73,13.86,...,0.5,0.77,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,None,NaN,33,163,NaN,7.52,38.42,...,0.5,1.00,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,None,NaN,16,95,NaN,5.61,24.67,...,0.5,0.00,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00


## Question 3. Counting short trips

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a `trip_distance` of less than or equal to 1 mile?

- 7,853
- 8,007 [x]
- 8,254
- 8,421

In [9]:
trip_df[
    (trip_df['lpep_pickup_datetime'] > '2025-11-01') &
    (trip_df['lpep_pickup_datetime'] <= '2025-12-01') &
    (trip_df['trip_distance'] <= 1)
].shape[0]

8007

## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Only consider trips with `trip_distance` less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

- 2025-11-14 [x]
- 2025-11-20
- 2025-11-23
- 2025-11-25

In [ ]:
trip_df['lpep_pickup_date'] = trip_df['lpep_pickup_datetime'].dt.date

In [13]:
trip_df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,lpep_pickup_date
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00,2025-11-01
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00,2025-11-01
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00,2025-11-01
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00,2025-11-01
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00,2025-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,None,NaN,59,51,NaN,8.50,33.22,...,0.00,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00,2025-11-30
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,None,NaN,74,151,NaN,1.73,13.86,...,0.77,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00,2025-11-30
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,None,NaN,33,163,NaN,7.52,38.42,...,1.00,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75,2025-11-30
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,None,NaN,16,95,NaN,5.61,24.67,...,0.00,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00,2025-11-30


In [15]:
trip_df[trip_df['trip_distance'] < 100].groupby('lpep_pickup_date')['trip_distance'].agg('max').sort_values(ascending=False)

lpep_pickup_date
2025-11-14    88.03
2025-11-20    73.84
2025-11-23    45.26
2025-11-22    40.16
2025-11-15    39.81
2025-11-11    39.00
2025-11-19    38.68
2025-11-16    37.67
2025-11-10    36.46
2025-11-26    36.10
2025-11-04    35.84
2025-11-28    34.85
2025-11-25    34.14
2025-11-29    33.08
2025-11-05    32.25
2025-11-08    32.08
2025-11-02    31.92
2025-11-24    31.83
2025-11-21    30.60
2025-11-09    29.05
2025-11-12    28.08
2025-11-30    27.83
2025-11-18    27.77
2025-11-01    27.69
2025-11-06    27.63
2025-11-13    27.58
2025-11-03    27.35
2025-11-07    27.11
2025-11-27    25.29
2025-11-17    22.44
2025-10-31    16.25
2025-10-30     5.70
2025-10-27     4.98
2025-12-01     4.81
2025-10-26     1.35
Name: trip_distance, dtype: float64

## Question 5. Biggest pickup zone

Which was the pickup zone with the largest `total_amount` (sum of all trips) on November 18th, 2025?

- East Harlem North [x]
- East Harlem South
- Morningside Heights
- Forest Hills

In [20]:
trip_df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee', 'lpep_pickup_date'],
      dtype='object')

In [34]:
import datetime

trip_df[trip_df['lpep_pickup_date'] == datetime.date(2025, 11, 18)].groupby('PULocationID')['total_amount'].agg('sum').sort_values(ascending=False)

PULocationID
74     9281.92
75     6696.13
43     2378.79
244    2139.05
166    2100.59
        ...   
77       16.00
249      16.00
151      16.00
32       13.57
34        0.00
Name: total_amount, Length: 137, dtype: float64

In [35]:
tzl_df[tzl_df['LocationID'] == 74]

,LocationID,Borough,Zone,service_zone
73,74,Manhattan,East Harlem North,Boro Zone


## Question 6. Largest tip

For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's `tip` , not `trip`. We need the name of the zone, not the ID.

- JFK Airport
- Yorkville West [x]
- East Harlem North
- LaGuardia Airport

In [37]:
trip_df['lpep_pickup_month'] = trip_df['lpep_pickup_datetime'].dt.month
trip_df['lpep_pickup_year'] = trip_df['lpep_pickup_datetime'].dt.year

In [39]:
trip_df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee', 'lpep_pickup_date', 'lpep_pickup_month',
       'lpep_pickup_year'],
      dtype='object')

In [40]:
trip_df['tip_amount']

0        1.94
1        0.00
2        5.00
3        0.50
4        1.00
         ... 
46907    0.00
46908    0.77
46909    1.00
46910    0.00
46911    3.89
Name: tip_amount, Length: 46912, dtype: float64

In [44]:
trip_df[
    (trip_df['PULocationID'] == 74) & 
    (trip_df['lpep_pickup_month'] == 11) & 
    (trip_df['lpep_pickup_year'] == 2025)
].groupby('DOLocationID')['tip_amount'].agg('sum').sort_values(ascending=False)

DOLocationID
236    4242.01
75     3425.94
238    2752.60
263    2403.17
166    2121.65
        ...   
144       0.00
67        0.00
76        0.00
45        0.00
83        0.00
Name: tip_amount, Length: 137, dtype: float64

In [46]:
tzl_df[tzl_df['LocationID'].isin([236, 263])]

,LocationID,Borough,Zone,service_zone
235,236,Manhattan,Upper East Side North,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone


## Question 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:
1. Downloading the provider plugins and setting up backend,
2. Generating proposed changes and auto-executing the plan
3. Remove all resources managed by terraform`

Answers:
- terraform import, terraform apply -y, terraform destroy
- teraform init, terraform plan -auto-apply, terraform rm
- terraform init, terraform run -auto-approve, terraform destroy
- terraform init, terraform apply -auto-approve, terraform destroy [x]
- terraform import, terraform apply -y, terraform rm